### Create new price list
(С всеми новыми тарифами в Combined)

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = 'c:/Users/balob/Documents/GITLAB/RB_BD/DATA/'
file_result = 'df_combined_with_new.csv'

eur_usd = 1.2

In [2]:
# Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE,t.NETWORK_ID,
np.MCC + IIF(LEN(MNC)=1,'0' + MNC,MNC) MCCMNC
FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES np ON t.network_id = np.network_id
'''
df_tadig_mccmnc = pd.read_sql_query(sql_srt, cnxn)
#df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
df_tadig = df_tadig_mccmnc[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','NETWORK_ID']].drop_duplicates()

df_tadig_mccmnc = df_tadig_mccmnc[['NETWORK_ID','MCCMNC']].drop_duplicates()

display(df_tadig.head())

cnxn.close()

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4.0
1,Afghanistan,Areeba/MTN,AFGAR,5.0
2,Afghanistan,Etisalat,AFG55,6.0
3,Afghanistan,Etisalat,AFGEA,6.0
4,Afghanistan,Roshan/TDCA,AFGK9,7.0


In [3]:
# Загружаем новые тарифы Partner и P4

df_partner = pd.read_csv(join(load_folder,'Partner_1_1_2019_for_tariffs.csv'),sep=';')
print('Тарифы Partner:')
display(df_partner.head())

df_p4 = pd.read_csv(join(load_folder,'P4_1_1_2019_for_tariffs.csv'),sep=';')
#df_p4[['MOC','MTC','SMS','MB']] = df_p4[['MOC','MTC','SMS','MB']]*eur_usd
print('Тарифы P4:')
display(df_p4.head())

Тарифы Partner:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
0,ALBVF,27602.0,Vodafone AL,Albania,VF,0.0439,0.0,0.0062,0.0030,1,1.0
1,ANDMA,21303.0,Andorra Telecom SAU,Andorra,NaN,0.2000,0.0,0.0300,0.2000,60,1.0
2,AIACW,36584.0,AIACW AI,Anguilla,C&W Lime,0.1606,0.0,0.0128,0.0257,60,1.0
3,ATGCW,34492.0,ATGCW AG,Antigua And Barbuda,C&W Lime,0.1500,0.0,0.0120,0.0240,60,1.0
4,ARGTM,72207.0,TELE MOV AR,Argentina,Telefonica LATAM,0.0728,0.0,0.0240,0.0180,1,1.0


Тарифы P4:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
0,Albania,ALBtelecom sh.a.,ALBEM,0.390,NaN,0.040,0.039,60,1
1,Anguilla,Cable and Wireless (Anguilla) Limited,AIACW,0.800,NaN,0.080,0.080,1,1
2,Antigua,Cable & Wireless Antigua and Barbuda Limited,ATGCW,0.800,NaN,0.080,0.080,1,1
3,Armenia,Armenia Telephone Company (ArmenTel),ARM01,0.585,NaN,0.015,0.004,60,1
4,Australia,SingTel Optus Pty Limited,AUSOP,0.055,NaN,0.008,0.008,60,1


In [4]:
# Загружаем Allowed list с необходимыми изменениями

df_al=pd.read_csv(join(load_folder,'DMI.dmi_allowed_list_export_Sun_Jan_13_2019.csv'),usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al = df_al[df_al.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al = df_al.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al = pd.merge(df_al,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al.drop('TADIG_CODE',inplace=True,axis=1)

print('Allowed list:')
display(df_al.head())

Allowed list:


,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,S2,HRVT2,225.0
1,S2,MNEMT,622.0
2,S1,MKDCC,569.0
3,S2,CZEET,234.0
4,S2,HRVVI,226.0


In [5]:
#Объединяем тарифы с TADIG и Allowed list

df_partner_net_id = pd.merge(df_partner,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах Partner с добавлением network_id:')
display(df_partner_net_id[df_partner_net_id.TADIG_CODE.isnull()])

df_p4_net_id = pd.merge(df_p4,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах P4 с добавлением network_id:')
display(df_p4_net_id[df_p4_net_id.TADIG_CODE.isnull()])

df_s1_s2_temp = pd.merge(df_partner_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                    df_p4_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                    how='outer',on=['NETWORK_ID'],suffixes=('_S1','_S2')).drop_duplicates()
#print('Тарифы Partner и P4:')
#display(df_s1_s2_temp.head())

df_s1_s2 = pd.merge(df_s1_s2_temp,df_tadig,how='left',on='NETWORK_ID')
#print('Тарифы Partner и P4 с TADIG:')
#display(df_s1_s2.head())

df_s1_s2_sponsor = pd.merge(df_s1_s2,df_al,how='outer',on='NETWORK_ID')
#print('Тарифы Partner и P4 с привязкой к Allowed list:')
#display(df_s1_s2_sponsor.head())

# Результирующий файл

#f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S1','MOC_S2','MTC_S1','MTC_S2','SMS_S1','SMS_S2','MB_S1','MB_S2','SPONSOR']
f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE',
          'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1',
          'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2',
          'SPONSOR','TADIG_CODE_AL','NETWORK_ID']
df_s1_s2_sponsor = df_s1_s2_sponsor[f_list].drop_duplicates()
df_s1_s2_sponsor.to_csv(join(load_folder,'Partner_P4_Compare_1_1_2019.csv'),index=False)
print('\nТарифы Partner и P4 с привязкой к Allowed list:')
df_s1_s2_sponsor.head()

Неизвестные TADIG в тарифах Partner с добавлением network_id:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
86,INDEU,40456.0,IdeaUP IN,India,IDEA,0.1,0.0,0.06,0.02,60,10.0,NaN,NaN,NaN,NaN
87,INDIB,40570.0,IDEA Bih IN,India,IDEA,0.1,0.0,0.06,0.02,60,10.0,NaN,NaN,NaN,NaN


Неизвестные TADIG в тарифах P4 с добавлением network_id:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
137,Mozambique,MOVITEL S.A,MOZVG,0.7,NaN,0.07,0.042,60,10,NaN,NaN,NaN,NaN



Тарифы Partner и P4 с привязкой к Allowed list:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,Voice_Increment_SEC_S2,Data_Increment_KB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,Albania,Vodafone,ALBVF,0.0439,0.0,0.0062,0.0030,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ALBVF,12.0
1,Andorra,Andorra Telecom S A U,ANDMA,0.2000,0.0,0.0300,0.2000,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,60.0,1.0,0.8,NaN,0.08,0.08,1.0,1.0,S1,AIACW,20.0
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,60.0,1.0,0.8,NaN,0.08,0.08,1.0,1.0,S1,ATGCW,23.0
4,Argentina,Movistar/Telefonica,ARG01,0.0728,0.0,0.0240,0.0180,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ARG01,27.0


In [6]:
# Финальная версия с MCCMNC

df_s1_s2_sponsor_mccmnc = pd.merge(df_s1_s2_sponsor,df_tadig_mccmnc,how='left',on='NETWORK_ID')
df_s1_s2_sponsor_mccmnc.head()

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,Voice_Increment_SEC_S2,Data_Increment_KB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID,MCCMNC
0,Albania,Vodafone,ALBVF,0.0439,0.0,0.0062,0.0030,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ALBVF,12.0,27602
1,Andorra,Andorra Telecom S A U,ANDMA,0.2000,0.0,0.0300,0.2000,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,21303
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,60.0,1.0,0.8,NaN,0.08,0.08,1.0,1.0,S1,AIACW,20.0,365840
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,60.0,1.0,0.8,NaN,0.08,0.08,1.0,1.0,S1,ATGCW,23.0,344920
4,Argentina,Movistar/Telefonica,ARG01,0.0728,0.0,0.0240,0.0180,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ARG01,27.0,72207


In [7]:
# Проверка наличия различных тарифов для одного и того же оператора.
# Операторы определяются по данным OCS

df = df_p4_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s1 = df[df.values > 1]
display(s1)
#df_s1_s2_sponsor[df_s1_s2_sponsor.NETWORK_ID.isin(s1.index.values)]

,MOC,MTC,SMS,MB
NETWORK_ID,,,,


In [8]:
# Подготовка Combined Price Plan

df1 = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC'
  ,'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1','SPONSOR','NETWORK_ID']].copy()

df1.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S1':'MOC','MTC_S1':'MTC','SMS_S1':'SMS','MB_S1':'DATA (MB)','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S1':'Voice Increment (SEC)','Data_Increment_KB_S1':'Data Increment (KB)'}
           ,axis=1,inplace=True)
df1['SPONSOR'] = 'S1'

df2 = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC'
  ,'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2','SPONSOR','NETWORK_ID']].copy()

df2.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S2':'MOC','MTC_S2':'MTC','SMS_S2':'SMS','MB_S2':'DATA (MB)','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S2':'Voice Increment (SEC)','Data_Increment_KB_S2':'Data Increment (KB)'}
           ,axis=1,inplace=True)
df2['SPONSOR'] = 'S2'

df2[['MOC','MTC','SMS','DATA (MB)']] = df2[['MOC','MTC','SMS','DATA (MB)']]*eur_usd

df_combined = pd.concat([df1, df2],axis=0).drop_duplicates().sort_values(by=['Country','Operator'])

df_combined.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all',inplace=True)

df_combined.MTC.fillna(0,inplace=True)

df_combined[['MOC','MTC','SMS','DATA (MB)']] = df_combined[['MOC','MTC','SMS','DATA (MB)']]*2

df_combined.loc[df_combined.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
= df_combined[df_combined.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_combined['CountrySponsor'] = df_combined.groupby('Country')['SPONSOR_AL'].transform('any')

df_combined = df_combined[df_combined.SPONSOR == df_combined.CountrySponsor]

df_combined.to_csv(join(load_folder,file_result),index=False)

df_combined.head()

,Country,Operator,TADIG,MCCMNC,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),SPONSOR_AL,NETWORK_ID,SPONSOR,CountrySponsor
0,Albania,Vodafone,ALBVF,27602,0.0878,0.0,0.0124,0.0060,1.0,1.0,S1,12.0,S1,S1
2,Anguilla,Cable and Wireless,AIACW,365840,0.3212,0.0,0.0256,0.0514,60.0,1.0,S1,20.0,S1,S1
3,Antigua and Barbuda,C & W,ATGCW,344920,0.3000,0.0,0.0240,0.0480,60.0,1.0,S1,23.0,S1,S1
4,Argentina,Movistar/Telefonica,ARG01,72207,0.1456,0.0,0.0480,0.0360,1.0,1.0,S1,27.0,S1,S1
5,Argentina,Movistar/Telefonica,ARG01,72210,0.1456,0.0,0.0480,0.0360,1.0,1.0,S1,27.0,S1,S1


In [15]:
list_new_countries = df_combined.loc[df_combined.SPONSOR_AL.isnull(), 'Country'].tolist()
df_combined.loc[df_combined.Country.isin(list_new_countries),
               ['Country','Operator','TADIG','MOC','MTC','SMS','DATA (MB)','SPONSOR_AL','CountrySponsor']].drop_duplicates().\
to_csv(join(load_folder,'new.csv'))

In [39]:
# Подготовка файла тарифов для загрузки в базу данных OCS для Real Combined Price Plan

# Файлы:
file_combined_100 = 'df_combined_100_with_new.csv'

df_combined_100 = df_combined[df_combined.SPONSOR_AL.isnull()] .copy()
df_combined_100.rename({'DATA (MB)':'DATA', 'SPONSOR':'SPONSOR_ID'},axis=1,inplace=True)
df_combined_100.SPONSOR_ID.replace(to_replace={'S1':1, 'S2':2}, inplace = True)

#Добавляем поля для удобства выгрузки
#!!! Поменять 'ROAMING_PLAN_ID', 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'

df_combined_100['ROAMING_PLAN_RULE_ID']=''
df_combined_100['ROAMING_PLAN_ID']=''
df_combined_100['MT_SMS_RATE']=0
df_combined_100['CURRENCY_ID']=1 #1:USD, 2:EUR
df_combined_100['START_DATE']='2019-01-01 00:00:00'
df_combined_100['END_DATE']='NULL'
df_combined_100['is_discounted']=1

df_combined_100[['TADIG',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC',
'MTC',
'SMS',
'MT_SMS_RATE',
'DATA',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',]].drop_duplicates().to_csv(join(load_folder,file_combined_100),index=False)